# Homework 1 - Francesco Brunello (M63001655), Antonio Boccarossa (M63001643)

---



# Introduction to Clinical Trials
Clinical trials are scientific studies conducted on human subjects to evaluate
the safety and effectiveness of medical treatments, drugs, devices, or procedures.
Each trial follows a defined protocol and may span several phases (Phase I, II,
III, IV), sometimes involving thousands of participants across multiple countries.

# The Dataset
The dataset used in this exercise comes from Dimensions.ai, a platform that
aggregates data on global scientific research. Each row in the dataset represents
a clinical trial; informations about the columns can be found in the provided
legend.csv file.
Some columns contain nested or structured data, such as lists of conditions,
organizations, or locations.

# Task
Perform at least five analytics using PySpark on the provided clinical trials
dataset. The results must be compiled and presented in a structured PDF
report. For each analysis, the report should include the following components:


*   Objective: The goal of the analysis.
*   Description: A brief description of the methodology used.
*   Code: The PySpark code used to perform the analysis.

Include analyses of varying complexity, from basic aggregations to more
complex operations.

### Dependencies
*Installing pyspark dependencies.*

In [1]:
# Run below commands in google colab
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.0
!wget -q http://apache.osuosl.org/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz
# unzip it
!tar xf spark-3.5.5-bin-hadoop3.tgz
# install findspark
!pip install -q findspark

### Spark Context and Session
*Creating Spark Context and Session in order to use pyspark.sql utilities*

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.5-bin-hadoop3"

In [3]:
# Verify the Spark version running on the virtual cluster
import pyspark as ps
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate()

assert  "3." in sc.version, "Verify that the cluster Spark's version is 3.x"

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)
print(spark)

### Reading the Dataset

*The given dataset is composed by various data entries related to clinical trials and it's composed by 15999 rows and 38 columns.*

In [5]:
import pandas as pd
df = pd.read_excel("dimensions_clinicalTrials.xlsx")
clinicalDS = spark.createDataFrame(df)
# printSchema shows the names and types of columns
clinicalDS.printSchema()

root
 |-- Rank: long (nullable = true)
 |-- Trial ID: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Brief title: string (nullable = true)
 |-- Acronym: string (nullable = true)
 |-- Abstract: string (nullable = true)
 |-- Start date: timestamp (nullable = true)
 |-- Start Year: double (nullable = true)
 |-- End Date: timestamp (nullable = true)
 |-- Completion Year: double (nullable = true)
 |-- Phase: string (nullable = true)
 |-- Study Type: string (nullable = true)
 |-- Study Design: string (nullable = true)
 |-- Conditions: string (nullable = true)
 |-- Recruitment Status: string (nullable = true)
 |-- Number of Participants: double (nullable = true)
 |-- Intervention: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Registry: string (nullable = true)
 |-- Investigators/Contacts: string (nullable = true)
 |-- Sponsors/Collaborators: string (nullable = true)
 |-- City of Sponsor/Collaborator: string (nullable

### **Objective N1: Find the number of studies started per year.**




In order to find the ***The number of studies started per year*** we grouped the rows by the column 'Start Year', counted the different studies identified by Trial ID for each year and sorted the result by the count.

In [83]:
from pyspark.sql.functions import count , desc

clinicalDS \
.select('Trial ID', 'Start Year') \
.distinct() \
.groupBy('Start Year') \
.agg(count('`Trial ID`') \
.alias('Number of Studies')) \
.orderBy(desc('Number of Studies')) \
.show(40)

+----------+-----------------+
|Start Year|Number of Studies|
+----------+-----------------+
|    2021.0|              722|
|    2020.0|              662|
|    2019.0|              640|
|    2018.0|              589|
|    2022.0|              585|
|    2017.0|              548|
|    2015.0|              457|
|    2016.0|              446|
|    2023.0|              399|
|    2014.0|              397|
|    2013.0|              370|
|    2012.0|              367|
|    2011.0|              325|
|    2010.0|              295|
|    2009.0|              287|
|    2008.0|              287|
|    2007.0|              226|
|    2006.0|              200|
|    2005.0|              130|
|    2004.0|              101|
|    2003.0|               56|
|    2001.0|               45|
|    2002.0|               40|
|    2024.0|               38|
|    2000.0|               35|
|       NaN|               25|
|    1998.0|               19|
|    1999.0|               18|
|    1997.0|               11|
|    199

### **Objective N2: Find the Top 10 most frequent medical conditions in all clinical trials**

In order to find ***The Top 10 most frequent medical conditions in all clinical trials*** we grouped the rows by the 'Conditions' field (exploding the list generated by the split function on the ";" and "," characters in order to consider all the effective values of Conditions field), counted and sorted all of them to find the results, filtering NaN values.






In [54]:
clinicalDS \
.select('Trial ID', 'Conditions') \
.distinct() \
.withColumn('Conditions', explode(split('Conditions', r"[;,]+"))) \
.withColumn('trimmedCondition', trim(col('Conditions'))) \
.filter(col('trimmedCondition') != 'NaN') \
.groupBy('trimmedCondition') \
.agg(count('trimmedCondition').alias('Entries')) \
.orderBy(desc('Entries')) \
.show(10, truncate=False)

+-----------------------+-------+
|trimmedCondition       |Entries|
+-----------------------+-------+
|Breast Cancer          |158    |
|Carcinoma              |97     |
|Lymphoma               |87     |
|Multiple Myeloma       |83     |
|Coronary Artery Disease|71     |
|Heart Failure          |67     |
|Lung Cancer            |63     |
|Leukemia               |62     |
|Ovarian Cancer         |60     |
|Colorectal Cancer      |56     |
+-----------------------+-------+
only showing top 10 rows



### **Objective N3: Find the Top 10 Funder Countries that financied the most relevant studies in terms of total Altmetric Attention Score.**

In order to find ***The Top 10 Funder Countries that financied the most relevant studies in terms of total Altmetric Attention Score*** we selected all the Countries (exploding the list generated by the split function on the ";" and "," characters in order to consider all the effective values of Funder Countries) and Altmetric Attention Score, filtering the NaN values, grouping by Countries and ordering the resultant rows by the total amount of the Total Altmetric Attention Score per Country.


In [86]:
from pyspark.sql.functions import explode, split, trim, sum, col

clinicalDS \
.select('Trial ID', explode(split('Funder Country', r"[;,]+")).alias('Countries'), 'Altmetric Attention Score') \
.distinct() \
.withColumn('Countries', trim(col("Countries"))) \
.filter(col('Altmetric Attention Score') != "NaN")\
.filter(col('Countries') != "NaN")\
.groupBy('Countries')\
.agg(sum('Altmetric Attention Score').alias('Total Altmetric Attention Score'))\
.orderBy(desc('Total Altmetric Attention Score')) \
.show(10)

+--------------+-------------------------------+
|     Countries|Total Altmetric Attention Score|
+--------------+-------------------------------+
| United States|                        90348.0|
|         Japan|                        16558.0|
|       Germany|                        12597.0|
|United Kingdom|                         4549.0|
|        France|                         2488.0|
|   Switzerland|                         1641.0|
|         Italy|                         1418.0|
|       Belgium|                          625.0|
|        Canada|                          449.0|
|   Netherlands|                          167.0|
+--------------+-------------------------------+
only showing top 10 rows



### **Objective N4: Find the Top 5 research fields with most clinic trials.**

In order to find ***The Top 5 fields of the research with most clinic trials*** we selected all the Fields of Research (exploding the list generated by the split function on the ";" and "," characters in order to consider all the effective values of Reasearch Fields) filtering the NaN values. Also, we grouped on the Research Field in order to count them.

In [87]:
clinicalDS\
.select('Trial ID', explode(split('Fields of Research (ANZSRC 2020)', r"[;,]+")).alias('Research Field')) \
.distinct() \
.withColumn('Research Field', trim(col("Research Field"))) \
.filter(col('Research Field') != "NaN") \
.groupBy('Research Field') \
.agg(count(col('Research Field')).alias('Count of trials')) \
.orderBy(desc('Count of trials')) \
.show(5, truncate=False)

+--------------------------------------------+---------------+
|Research Field                              |Count of trials|
+--------------------------------------------+---------------+
|32 Biomedical and Clinical Sciences         |7860           |
|3202 Clinical Sciences                      |4144           |
|3211 Oncology and Carcinogenesis            |2492           |
|42 Health Sciences                          |835            |
|3201 Cardiovascular Medicine and Haematology|750            |
+--------------------------------------------+---------------+
only showing top 5 rows



### **Objective N5: Find the longest Phase 3 of Clinical Trials in term of expected or actual years.**




In order to find ***The longest Phase 3 of Clinical Trials in term of expected or actual years*** we created a new column called 'Years' filling it with the difference between 'Completion Year' and 'Start Year' (previously filtered to avoid NaN values). In addition, we selected the appropriate fields, filtering the output on 'Phase' field and ordering all by 'Years' field.

In [119]:
clinicalDS \
.filter(col('Completion Year') != 'NaN') \
.filter(col('Start Year') != 'NaN') \
.withColumn('Years', col('Completion Year') - col('Start Year')) \
.select('Trial ID', 'Abstract', 'Years', 'Completion Year', 'Start Year') \
.distinct() \
.where(col('Phase') == 'Phase 3') \
.orderBy(desc('Years')) \
.show(2)

+-----------+--------------------+-----+---------------+----------+
|   Trial ID|            Abstract|Years|Completion Year|Start Year|
+-----------+--------------------+-----+---------------+----------+
|NCT00070564|RATIONALE: Drugs ...| 24.0|         2027.0|    2003.0|
|NCT01704716|This is a randomi...| 24.0|         2026.0|    2002.0|
+-----------+--------------------+-----+---------------+----------+
only showing top 2 rows



### **Objective N6: Find the most frequent HRCS HC Categories of study per gender (All, Female, Male), studied between 2005 and 2021 ordered by the sum of the Altmetric Attention Score of the related clinical trials.**




In order to find ***The most frequent HRCS HC Categories of study per gender (All, Female, Male), studied between 2005 and 2021 ordered by the sum of the Altmetric Attention Score of the related clinical trials*** we created a Window object in order to partitition the final output by the Gender besed on the rank assigned by the row_number function. This function, associates a "rank" value based on the Total Attention Score assumed by that istance.

In the query, we selected the HRCS HC Categories (exploding the list generated by the split function on the ";" and "," characters in order to consider all the effective values of HRCS HC Categories field), filtering the data by years and removing the NaN values. Also, we grouped the data by Gender and Categories, in order to execute the sum of the AASs values and find the first ranked row for each gender.

In [89]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

windowSpec = Window.partitionBy("Gender").orderBy(desc("Total Attention Score")) #Partizioniamo in base al genere e ordiniamo in base al Total Attention Score

results = clinicalDS.select('Trial ID', explode(split('HRCS HC Categories', r'[;,]+')).alias('Categories'), 'Gender', 'Altmetric Attention Score') \
.distinct() \
.withColumn('Categories', trim(col('Categories'))) \
.filter(col('Categories')!= 'NaN') \
.filter(col('Gender') != 'NaN') \
.filter(col('Altmetric Attention Score') != 'NaN') \
.filter(col('Start Year')> 2004) \
.filter(col('Completion Year')< 2022) \
.groupBy('Gender', 'Categories' ) \
.agg(sum('Altmetric Attention Score').alias('Total Attention Score')) \
.orderBy(desc('Total Attention Score')) \
.withColumn("rank", row_number().over(windowSpec)) \
.filter(col("rank") == 1) \
.drop("rank") \
.show()


+------+----------+---------------------+
|Gender|Categories|Total Attention Score|
+------+----------+---------------------+
|   All|    Cancer|              11539.0|
|Female|    Cancer|                780.0|
|  Male|    Cancer|                141.0|
+------+----------+---------------------+



### **Objective N7: Find the Top 10 AHC involved in Mental Health Category clinical trials with more than 100 participants.**


In order to find ***The Top 10 AHC involved in Mental Health category clinical trials with more then 100 participants*** we selected the HRCS HS Categories (exploding the list generated by the split function on the ";" and "," characters in order to consider all the effective values of HRCS HS Categories) filterintg NaN values and finding only the rows with 'Mental Health' category. We also filtered the data by the Number of Partecipants, grouped by AHC values and ordered by Number of trails in order to show the right output.

In [90]:
clinicalDS.select('Trial ID', 'AHC', explode(split('HRCS HC Categories', r'[;,]+')).alias('HRCS HC Categories Splitted') ) \
.distinct() \
.withColumn('HRCS HC Categories Splitted', trim(col('HRCS HC Categories Splitted'))) \
.filter(col('HRCS HC Categories Splitted') != 'NaN') \
.filter(col('HRCS HC Categories Splitted') == 'Mental health') \
.filter(col('Number of Participants') > 100) \
.groupBy('AHC')\
.agg(count('AHC').alias('Number of trials')) \
.orderBy(desc('Number of trials')) \
.show(10, truncate=False)

+---------------------+----------------+
|AHC                  |Number of trials|
+---------------------+----------------+
|IRCCS_CAGRANDA       |8               |
|AOU_SENESE           |7               |
|AOU_CITTADELLASCIENZA|6               |
|AOUSSN_GMARTINO      |6               |
|AOU_CAREGGI          |5               |
|IRCCS_BURLOGAROFOLO  |5               |
|AOU_VERONA           |4               |
|AOU_RIUNITIFOGGIA    |3               |
|IRCCS_SANMARTINO     |3               |
|AOU_GONZAGA          |3               |
+---------------------+----------------+
only showing top 10 rows



### **Objective N8: Find the most used Intervention Models (in Study Design field) and the most used couple Intervention Models/Intervention Method (in Intervention field) in cancer related clinical trails**

In order to find ***The most used Intervention Models (in Study Design field) and the most used couple Intervention Models/Intervention Method (in Intervention field) in cancer related clinical trails*** we made two separate queries.
In the first one, we selected the Study Design categories (exploding the list generated by the split function on the ";" and "," characters in order to consider all the effective values of Study Design) filterintg NaN values and finding only the rows with 'Intervention' value. We also grouped the data by the new field Design Phases and ordered the data by the Number of Intevention in order to show the correct output.

In the second one, we selected the Study Design categories (exploding the list generated by the split function on the ";" and "," characters in order to consider all the effective values of Study Design) and Intevention in the same way. In addittion, thanks to a window object we ranked the couples Intervention Models/Intervention previously and filtered only the first ranked entries.


In [105]:
# First Query
print("FIRST QUERY")
clinicalDS.select('Trial ID', explode(split('Study Design', r'[;,]+')).alias('Design Phases'))\
.distinct() \
.withColumn('Design Phases', trim(col('Design Phases')))\
.filter(col('Design Phases').startswith('Intervention'))\
.filter(col('Cancer Types') != 'NaN') \
.groupBy('Design Phases')\
.agg(count('Design Phases').alias('Number of Intervention'))\
.orderBy(desc('Number of Intervention'))\
.show(6, truncate=False)


#Second Query
print("SECOND QUERY")
windowSpec = Window.partitionBy('Design Phases').orderBy(desc('Number of Intervention'))

clinicalDS.select('Trial ID', explode(split('Study Design', r'[;,]+')).alias('Design Phases'), explode(split('Intervention', r'[;,]+')).alias('Intervention Method'))\
.distinct() \
.withColumn('Design Phases', trim(col('Design Phases')))\
.withColumn('Intervention Method', trim(col('Intervention Method')))\
.filter(col('Design Phases').startswith('Intervention'))\
.filter(col('Cancer Types') != 'NaN') \
.filter(col('Intervention Method') != 'NaN') \
.groupBy('Design Phases', 'Intervention Method')\
.agg(count('Design Phases').alias('Number of Intervention'))\
.withColumn("rank", row_number().over(windowSpec)) \
.filter(col("rank") == 1) \
.drop("rank") \
.orderBy(desc('Number of Intervention'))\
.show(truncate=False)

FIRST QUERY
+-------------------------------------------+----------------------+
|Design Phases                              |Number of Intervention|
+-------------------------------------------+----------------------+
|Intervention Model: Parallel Assignment    |1440                  |
|Intervention Model: Single Group Assignment|646                   |
|Intervention Model: Sequential Assignment  |98                    |
|Intervention Model: Crossover Assignment   |37                    |
|Intervention Model: Factorial Assignment   |18                    |
|Intervention Model: Cohort Study           |2                     |
+-------------------------------------------+----------------------+
only showing top 6 rows

SECOND QUERY
+-------------------------------------------+-------------------------------------+----------------------+
|Design Phases                              |Intervention Method                  |Number of Intervention|
+-------------------------------------------+-

### **Objective N9: Found the most frequent Investigator per Study Type started between 2005 and 2025**






In order to find ***The most frequent Investigator per Study Type started between 2005 and 2025*** we grouped on Study Type and Investigators on the filtered fields in order to count how many times an Investigator took part in a specific Study Type, per every Study Type. In order to got the right output, we used also in this case a Window in order to filter the output data.





In [116]:

windowSpec=Window.partitionBy('Study Type').orderBy(desc('Number of Studies'))

clinicalDS.select('Trial ID', 'Study Type', explode(split(trim(col('Investigators/Contacts')), r"[;,]+")).alias('Investigators'), 'Start Year') \
.withColumn('Investigators', trim(col('Investigators'))) \
.distinct() \
.filter(col('Start Year') > 2004) \
.filter(col('Start Year') < 2026) \
.filter(col('Start Year') != 'NaN') \
.filter(col('Investigators') != 'NaN') \
.groupBy('Study Type', 'Investigators') \
.agg(count('Study Type').alias('Number of Studies')) \
.withColumn("rank", row_number().over(windowSpec)) \
.filter(col("rank") == 1) \
.drop("rank") \
.orderBy(desc('Number of Studies')) \
.show(truncate=False)


+-------------------+------------------+-----------------+
|Study Type         |Investigators     |Number of Studies|
+-------------------+------------------+-----------------+
|Interventional     |ABBVIE INC        |57               |
|Observational      |Davide Chiumello  |12               |
|Non-interventional |Claudio Franceschi|2                |
|Active surveillance|Leone Roberto     |1                |
|CCT                |Carolyn Hoppe     |1                |
|Other              |Marco  Scioscia   |1                |
|RCT                |Soccoh Alex Kabia |1                |
+-------------------+------------------+-----------------+



### **Objective N10: Find the Top 5 Sponsor States that are most active in Breast Cancer (Condition) clinical trials started from 2010 until nowdays.**

In order to find ***The Top 5 Sponsor States that are most active in Breast Cancer (Condition) clinical trials from 2010 until nowdays*** we grouped the data (previously exploded, splitted and filtered) on 'States' and counted the Trials IDs of clinical trials that specifies Breast Cancer in 'Condition' field and. We also ordered the output by the Clinical Trials count in Breast Cancer.


In [140]:
from pyspark.sql.functions import year, current_date

clinicalDS.select('Trial ID', explode(split(col('State of Sponsor/Collaborator'), r"[;,]+")).alias('States'), 'Start Year',
                  explode(split( col('Conditions'), r"[;,]+" )).alias('Conditions')) \
                  .distinct() \
                  .withColumn('States', trim(col('States'))) \
                  .withColumn('Conditions', trim(col('Conditions'))) \
                  .filter(col('States')!="") \
                  .filter(col('States')!=" ") \
                  .filter(col('Conditions') == 'Breast Cancer') \
                  .filter(col('Start Year') > 2010 ) \
                  .filter(col('Start Year') < year(current_date()+1)) \
                  .groupBy('States') \
                  .agg(count('Trial ID').alias('Clinical Trials count in Breast Cancer')) \
                  .orderBy(desc('Clinical Trials count in Breast Cancer')) \
                  .show(5, truncate=False)


+----------+--------------------------------------+
|States    |Clinical Trials count in Breast Cancer|
+----------+--------------------------------------+
|California|52                                    |
|New York  |39                                    |
|Texas     |30                                    |
|Florida   |30                                    |
|New Jersey|29                                    |
+----------+--------------------------------------+
only showing top 5 rows



## **Objective N11: Average number of participants per study title**

In order to find The **Average number of participants per study title** we selected the **Title** category and then calculate the average of the **Number of Participants** on that field.


In [ ]:
clinicalDS.select("Trial ID", "Title", "Number of Participants") \
.distinct() \
.filter(col("Number of Participants")!="NaN") \
.groupBy("Title") \
.avg("Number of Participants") \
.orderBy(desc("avg(Number of Participants)")) \
.show()

+--------------------+---------------------------+
|               Title|avg(Number of Participants)|
+--------------------+---------------------------+
|The epidemiology,...|                  2400000.0|
|ACCESS template p...|                  1000000.0|
|National Database...|                   285600.0|
|Computerized Regi...|                   120000.0|
|Effect of Pharmac...|                   115000.0|
|CRICKET: Critical...|                   105000.0|
|Digital Strategie...|                    82800.0|
|RICALOR - Registr...|                    63692.0|
|Analysis of In-ho...|                    60000.0|
|What is the Best ...|                    60000.0|
|Cohort Event Moni...|                    60000.0|
|HPV as Primary Sc...|                    60000.0|
|International Ran...|                    53142.0|
|PRospective Evalu...|                    52156.0|
|DianaWeb: Before ...|                    50000.0|
|A Multicenter, Cr...|                    39480.0|
|Evaluation and Im...|         

## **Objective N12: Countries with the highest average number of participants per study**

In order to find The **Countries with the highest average number of participants per study** we selected the **Country of Sponsor/Collaborator, Trial ID and Number of Participants** Categories (exploding the Country of Sponsor/Collaborator list generated by the split function on the ";" character in order to consider all the effective values) filterintg NaN values on **Number of Participants** field and calculating the average on it. Note: we used the *trim* function in order to remove the spaces between each word.

In [ ]:
clinicalDS.select(explode(split("Country of Sponsor/Collaborator",";")).alias("Countries"),"Trial ID", "Number of Participants") \
.distinct() \
.withColumn("Countries", trim(col("Countries"))) \
.filter((col("Countries").isNotNull()) & (col("Countries") != "")) \
.groupBy("Trial ID", "Countries") \
.avg("Number of Participants") \
.filter(col("avg(Number of Participants)") != "NaN") \
.orderBy(desc("avg(Number of Participants)")).show(100)

+--------------+--------------------+---------------------------+
|      Trial ID|           Countries|avg(Number of Participants)|
+--------------+--------------------+---------------------------+
|ISRCTN91495258|            Slovenia|                  2400000.0|
|ISRCTN91495258|      United Kingdom|                  2400000.0|
|ISRCTN91495258|              France|                  2400000.0|
|ISRCTN91495258|            Portugal|                  2400000.0|
|ISRCTN91495258|             Ireland|                  2400000.0|
|ISRCTN91495258|              Sweden|                  2400000.0|
|ISRCTN91495258|             Hungary|                  2400000.0|
|ISRCTN91495258|               Italy|                  2400000.0|
|ISRCTN91495258|           Lithuania|                  2400000.0|
|ISRCTN91495258|              Turkey|                  2400000.0|
|ISRCTN91495258|         Netherlands|                  2400000.0|
|ISRCTN91495258|              Latvia|                  2400000.0|
|ISRCTN914

## **Objective N13: Find the 10 most popular cities of sponsor/collaborator per study type**

In order to find The **10 most popular cities of sponsor/collaborator per study type** we selected the **City of Sponsor/Collaborator, Trial ID and Study Type** Categories (exploding the City of Sponsor/Collaborator list generated by the split function on the ";" character in order to consider all the effective values) grouping by "Study Type" and **Collaborator_Cities** fields and counting the number of trials for each city.

In [ ]:
clinicalDS.select(explode(split("City of Sponsor/Collaborator", "; ")).alias("Collaborator_Cities"), "Study Type", "Trial ID") \
.distinct() \
.withColumn("Collaborator_Cities", trim(col("Collaborator_Cities"))) \
.filter((col("Collaborator_Cities").isNotNull()) & (col("Collaborator_Cities") != "")) \
.groupBy("Study Type", "Collaborator_Cities") \
.agg(count(clinicalDS["Trial ID"]).alias("Numero_Prove")).orderBy(desc("Numero_Prove")).show(10)


+--------------+-------------------+------------+
|    Study Type|Collaborator_Cities|Numero_Prove|
+--------------+-------------------+------------+
|Interventional|              Milan|        2585|
|Interventional|               Rome|        2080|
|Interventional|          Barcelona|        2010|
|Interventional|             Madrid|        1915|
|Interventional|             London|        1751|
|Interventional|              Paris|        1445|
|Interventional|           New York|        1442|
|Interventional|            Bologna|        1333|
|Interventional|            Houston|        1115|
|Interventional|             Naples|        1112|
+--------------+-------------------+------------+
only showing top 10 rows



## **Objective N14: Find the clinical trial with the highest Altmetric Attention Score that started after July 10, 2013**

In order to find The **the clinical trial with the highest Altmetric Attention Score that started after July 10, 2013** we selected the **Trial ID and Altmetric Attention Score** Categories, ordering by "Altmetric Attention Score" field and choosing two conditions for the *select* clause, according to the query.

In [ ]:
clinicalDS.select("Trial ID", "Altmetric Attention Score") \
.orderBy(desc(clinicalDS["Altmetric Attention Score"])) \
.where((clinicalDS["Start Date"] > "2013-07-10") & (clinicalDS["Altmetric Attention Score"]!="NaN")) \
.limit(1).show()

+-----------+-------------------------+
|   Trial ID|Altmetric Attention Score|
+-----------+-------------------------+
|NCT04575597|                   1703.0|
+-----------+-------------------------+



## **Objective N15: Find the 5 most relevant (based on number of participants) trials per Cancer Types**

In order to find The **the 5 most relevant (based on number of participants) trials per Cancer Types** we selected the **Cancer Types, Trial ID and Number of Participants** Categories (exploding the Cancer Types list generated by the split function on the ";" character in order to consider all the effective values), ordering by "Number of Participants" field and filtering on "Cancer Types" and "Number of Participants" field to show the output properly.

In [ ]:
clinicalDS.select(explode(split("Cancer Types", "; ")).alias("Tipi_Cancro"), "Trial ID", "Number of Participants") \
.distinct() \
.withColumn("Tipi_Cancro", trim(col("Tipi_Cancro"))) \
.filter((col("Tipi_Cancro").isNotNull()) & (col("Tipi_Cancro") != "") & (col("Tipi_Cancro") != "NaN")) \
.filter(col("Number of Participants")!="NaN") \
.orderBy(desc(clinicalDS["Number of Participants"])).limit(5).show()

+--------------------+-----------+----------------------+
|         Tipi_Cancro|   Trial ID|Number of Participants|
+--------------------+-----------+----------------------+
|Not Site-Specific...|NCT05339841|               82800.0|
|Not Site-Specific...|NCT02038491|               63692.0|
|       Breast Cancer|NCT04590560|               60000.0|
|     Cervical Cancer|NCT01837693|               60000.0|
|Genital System, F...|NCT01837693|               60000.0|
+--------------------+-----------+----------------------+



## **Objective N16: Retrieve the start date and the completion year of the top 10 clinical trials that involve the "AOUSSN_CAGLIARI" AHC and with the highest Altmetric Attention Score per condition**

In order to find The **start date, the completion year and studied conditions of the top 10 clinical trials that involve the "AOUSSN_CAGLIARI" AHC and with the highest Altmetric Attention Score** we selected the **Trial ID, Start Date, Completion Year, Conditions, AHC and Altmetric Attention Score** Categories, ordering by "Altmetric Attention Score" field, filtering on the same field and filtering on the "AHC" field to show the output properly.

In [ ]:
clinicalDS.select("Trial ID","Start Date", "Completion Year",explode(split("Conditions",r"[;,]+")).alias("Conditions"),"AHC", "Altmetric Attention Score") \
.distinct() \
.filter(col("Altmetric Attention Score") != "NaN") \
.where(clinicalDS["AHC"]=="AOUSSN_CAGLIARI") \
.groupBy("Conditions","Trial ID","Start Date", "Completion Year","Conditions","AHC", "Altmetric Attention Score") \
.agg(max(clinicalDS["Altmetric Attention Score"])) \
.orderBy(desc("max(Altmetric Attention Score)")) \
.show(10)

+--------------------+-----------+-------------------+---------------+--------------------+---------------+-------------------------+------------------------------+
|          Conditions|   Trial ID|         Start Date|Completion Year|          Conditions|            AHC|Altmetric Attention Score|max(Altmetric Attention Score)|
+--------------------+-----------+-------------------+---------------+--------------------+---------------+-------------------------+------------------------------+
| G12c Mutated /Ad...|NCT04303780|2020-06-04 00:00:00|         2026.0| G12c Mutated /Ad...|AOUSSN_CAGLIARI|                    788.0|                         788.0|
|              KRAS p|NCT04303780|2020-06-04 00:00:00|         2026.0|              KRAS p|AOUSSN_CAGLIARI|                    788.0|                         788.0|
|     Crohn's Disease|NCT03466411|2018-04-13 00:00:00|         2030.0|     Crohn's Disease|AOUSSN_CAGLIARI|                    693.0|                         693.0|
|      Hep

## **Objective N17: Find all the clinical trials started after 2022 with at most 250 participants per Country**

In order to find The **Find all the clinical trials started after 2022 with at most 250 participants and group by conditions** we added two columns: one for the minimum age for each trial and the second for the maximum age and we specified a regular expression to retrieve the numeric values properly. After that, we added one more column to show the differences between each maximum and minimum and finally we selected **Trial ID and diff_age** for the output, ordering on the "diff_age" field.

In [ ]:
clinicalDS.select("Trial ID","Start Year", explode(split("Country of Sponsor/Collaborator",r"[;,]+")).alias("Paesi")) \
.distinct() \
.where((clinicalDS["Number of Participants"]>=250) & (clinicalDS["Start Year"]>"2022")) \
.groupBy("Paesi") \
.agg(count("Trial ID").alias("Numero_Prove")) \
.orderBy(desc("Numero_Prove")) \
.show()

+---------------+------------+
|          Paesi|Numero_Prove|
+---------------+------------+
|          Italy|          51|
|  United States|          29|
|  United States|          23|
|          Spain|          22|
|        Germany|          19|
|         Canada|          18|
|         France|          18|
|         Poland|          15|
|        Belgium|          14|
| United Kingdom|          14|
|      Australia|          13|
|          Italy|          13|
|    Netherlands|          13|
|               |          13|
|        Hungary|          12|
|         Taiwan|          11|
|         Israel|          11|
|    Switzerland|          11|
|        Austria|          11|
|          Japan|          10|
+---------------+------------+
only showing top 20 rows



## **Objective N18: Find the most popular (by number of trials) Field of Research per country with at least 500 participants**

In order to find The **most popular (by number of trials) Field of Research per country with at least 500 participants**, we created a window in order to create a partition on the "Country of Sponsor/Collaborator" (ordering by the number of trials) and we added one column in order to explode the **Fields of Research** field properly and one column for the **Country of Sponsor/Collaborator** (the process for this column is exactly the same we did for the first one). After that, we added one **where** clause on the "Number of Participants", we filtered on the same field to display the output properly. Finally grouped by the two newly columns and we counted (and ordered) on the number of trials.

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

windowSpec = Window.partitionBy("Paesi").orderBy(desc("Numero_Prove"))

clinicalDS.select("Fields of Research (ANZSRC 2020)","Country of Sponsor/Collaborator","Trial ID") \
.distinct() \
.withColumn("Campi_Ricerca", explode(split("Fields of Research (ANZSRC 2020)",r"[;,]+"))) \
.withColumn("Paesi", explode(split("Country of Sponsor/Collaborator",";"))) \
.where(clinicalDS["Number of Participants"]>=500) \
.withColumn("Paesi",trim(col("Paesi"))) \
.filter(col("Number of Participants")!="NaN") \
.filter((col("Paesi") != "") & (col("Campi_Ricerca") != "")) \
.groupBy("Paesi", "Campi_Ricerca") \
.agg(count("Trial ID").alias("Numero_Prove")) \
.withColumn("rank", row_number().over(windowSpec)) \
.filter(col("rank") == 1) \
.orderBy(desc("Numero_Prove")) \
.drop("rank") \
.show(truncate=False)

+------------------+-----------------------------------+------------+
|Paesi             |Campi_Ricerca                      |Numero_Prove|
+------------------+-----------------------------------+------------+
|United States     |32 Biomedical and Clinical Sciences|55467       |
|Italy             |32 Biomedical and Clinical Sciences|15046       |
|Germany           |32 Biomedical and Clinical Sciences|8909        |
|Spain             |32 Biomedical and Clinical Sciences|8136        |
|France            |32 Biomedical and Clinical Sciences|7857        |
|Japan             |32 Biomedical and Clinical Sciences|6391        |
|United Kingdom    |32 Biomedical and Clinical Sciences|5405        |
|China             |32 Biomedical and Clinical Sciences|5131        |
|Canada            |32 Biomedical and Clinical Sciences|4468        |
|Poland            |32 Biomedical and Clinical Sciences|3976        |
|Australia         |32 Biomedical and Clinical Sciences|3806        |
|Belgium           |

## **Objective N19: Find the most popular cities with a recruitment status different from "Completed" that have a completion year after 2027, grouping by Study Type**

In order to find The **the most popular cities with a recruitment status different from "Completed" that have a completion year after 2027, grouping by Study Type** we selected the **Trial ID, Completion Year, Study Type and City of Sponsor/Collaborator** Categories (exploding the _City of Sponsor/Collaborator_ list generated by the split function on the ";" character in order to consider all the effective values), grouping by ""Study Type", "Completion Year", "Trial ID"" fields and ordering by the number of trials.

In [ ]:
clinicalDS.select("Trial ID",explode(split("City of Sponsor/Collaborator",r"[;,]+").alias("Cities")),"Completion Year","Study Type") \
.distinct() \
.where((clinicalDS["Recruitment Status"]!="Completed") & (clinicalDS["Completion Year"]>2027)) \
.groupBy("Study Type", "Completion Year", "Trial ID") \
.agg(count("Trial ID").alias("Numero_Prove")) \
.orderBy(desc("Numero_Prove")) \
.show()

+--------------+---------------+-----------+------------+
|    Study Type|Completion Year|   Trial ID|Numero_Prove|
+--------------+---------------+-----------+------------+
|Interventional|         2029.0|NCT04173273|         378|
| Observational|         2030.0|NCT01848561|         372|
|Interventional|         2041.0|NCT04852887|         351|
|Interventional|         2028.0|NCT03398135|         331|
|Interventional|         2030.0|NCT05242471|         295|
|Interventional|         2029.0|NCT05242484|         258|
|Interventional|         2028.0|NCT04524611|         242|
|Interventional|         2030.0|NCT04035005|         169|
|Interventional|         2030.0|NCT03568318|         166|
|Interventional|         2030.0|NCT05578976|         165|
|Interventional|         2028.0|NCT02308085|         164|
|Interventional|         2029.0|NCT04595565|         150|
|Interventional|         2028.0|NCT01844986|         147|
|Interventional|         2029.0|NCT04548999|         138|
| Observationa

## **Objective N20: Find the 5 top AHCs that have sponsored the most number of trials about Pneumonia condition since 2020**

In order to find **the 5 top AHCs that have sponsored the most number of trials about polmonite since 2020** we selected the **AHC, Trial ID, Conditions and Start Year** Categories (exploding the _Conditions_ list generated by the split function on the ";" character in order to consider all the effective values). After that, we detemined a _where_ clause on "Conditions" and "Start Year" in order to display the proper output and grouping by "AHC", "Conditions", "Start Year" fields and ordering by the number of trials. Finally, we limited the output to only 5 entries of the dataset.

In [ ]:
clinicalDS.select("AHC","Trial ID",explode(split("Conditions",r"[;,]+")).alias("Conditions"), "Start Year") \
.distinct() \
.where((col("Conditions")=="Pneumonia") & (col("Start Year")>=2020)) \
.groupBy("AHC", "Conditions", "Start Year") \
.agg(count("Trial ID").alias("Numero_Prove")) \
.orderBy(desc("Numero_Prove")) \
.limit(5) \
.show()

+----------------+----------+----------+------------+
|             AHC|Conditions|Start Year|Numero_Prove|
+----------------+----------+----------+------------+
|IRCCS_SANGERARDO| Pneumonia|    2021.0|           1|
|IRCCS_SANTORSOLA| Pneumonia|    2021.0|           1|
| AOUSSN_UMBERTOI| Pneumonia|    2021.0|           1|
|IRCCS_SANMARTINO| Pneumonia|    2021.0|           1|
+----------------+----------+----------+------------+

